# Child Mind Institute — Problematic Internet Use



## Team notes & log



🔔 <b>REMINDER:</b> Make sure to use GitHub pull before starting work and push when finishing.



<u><b>To Do:</b></u>



- Explore each category of data to see how it should be handled.

- CP: Handle missing values. (Idea: Use KNN)

- AG: Drop unnecessary columns.  Dropped a few of them - AG 11/7- Drop few more 

- One-hot encoding, where possible.

- CP: Start training the model (Random Forest).
- AG : Visualization
- Finish Due Date 11/12/24. Before 6 pm Tuesday 


<u>Notes from Célie</u>



- 11/2: Hi Anusha! I will label my work with a comment and my initials (CP).

- 11/2: You can link the Kaggle notebook to your GitHub from within Kaggle.

- 11/3: I have set up this notebook so it works either locally or via Kaggle.

- 11/3: Cleaned up some repetitve sections.

- 11/5: Merged notebooks with AG & CP's work.


## Preprocessing


In [4]:
# Import libraries



# CP

import numpy as np

import pandas as pd

#### AG 11/2/24 ###

import matplotlib.pyplot as plt 

import seaborn as sns 

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

In [5]:
import os



# CP: Load data



# CP: Check if you are running in Kaggle or locally



#CP: Running locally

if os.path.exists('kaggle_data'):

    train = pd.read_csv('kaggle_data/train.csv')

    test = pd.read_csv('kaggle_data/test.csv')

    data_dict = pd.read_csv('kaggle_data/data_dictionary.csv')



# CP: Running in Kaggle

else:

    train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')

    test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')

    data_dict = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv')



# CP: Show all rows when displaying data

pd.set_option('display.max_rows', None)



# CP: Display data dictionary

data_dict

,Instrument,Field,Description,Type,Values,Value Labels
0,Identifier,id,Participant's ID,str,NaN,NaN
1,Demographics,Basic_Demos-Enroll_Season,Season of enrollment,str,"Spring, Summer, Fall, Winter",NaN
2,Demographics,Basic_Demos-Age,Age of participant,float,NaN,NaN
3,Demographics,Basic_Demos-Sex,Sex of participant,categorical int,"0,1","0=Male, 1=Female"
4,Children's Global Assessment Scale,CGAS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
5,Children's Global Assessment Scale,CGAS-CGAS_Score,Children's Global Assessment Scale Score,int,NaN,NaN
6,Physical Measures,Physical-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
7,Physical Measures,Physical-BMI,Body Mass Index (kg/m^2),float,NaN,NaN
8,Physical Measures,Physical-Height,Height (in),float,NaN,NaN
9,Physical Measures,Physical-Weight,Weight (lbs),float,NaN,NaN


In [6]:
#### AG 11/2/24 ###

train.head(20)

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.00,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.00,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.50,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.00,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,001f3379,Spring,13,1,Winter,50.0,Summer,22.279952,59.50,112.2,...,1.0,2.0,1.0,34.0,Summer,40.0,56.0,Spring,0.0,1.0
6,0038ba98,Fall,10,0,NaN,NaN,Fall,19.660760,55.00,84.6,...,4.0,1.0,0.0,20.0,Winter,27.0,40.0,Fall,3.0,0.0
7,0068a485,Fall,10,1,NaN,NaN,Fall,16.861286,59.25,84.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,2.0,NaN
8,0069fbed,Summer,15,0,NaN,NaN,Spring,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.0,NaN
9,0083e397,Summer,19,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#There are null values in every column except for "age" , "sex", and "basic season" 

#sii: Suicide Ideation Inventory: Range (0-3)

train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Basic_Demos-Age,3960.0,10.433586,3.574648,5.000000,8.000000,10.000000,13.000000,22.000000
Basic_Demos-Sex,3960.0,0.372727,0.483591,0.000000,0.000000,0.000000,1.000000,1.000000
CGAS-CGAS_Score,2421.0,65.454771,22.341862,25.000000,59.000000,65.000000,75.000000,999.000000
Physical-BMI,3022.0,19.331929,5.113934,0.000000,15.869350,17.937682,21.571244,59.132048
Physical-Height,3027.0,55.946713,7.473764,33.000000,50.000000,55.000000,62.000000,78.500000
Physical-Weight,3076.0,89.038615,44.569040,0.000000,57.200000,77.000000,113.800000,315.000000
Physical-Waist_Circumference,898.0,27.278508,5.567287,18.000000,23.000000,26.000000,30.000000,50.000000
Physical-Diastolic_BP,2954.0,69.648951,13.611226,0.000000,61.000000,68.000000,76.000000,179.000000
Physical-HeartRate,2967.0,81.597236,13.665196,27.000000,72.000000,81.000000,90.500000,138.000000
Physical-Systolic_BP,2954.0,116.983074,17.061225,0.000000,107.000000,114.000000,125.000000,203.000000


In [8]:
####AG###



#### Notes ####


## Children Global Assessment Scale(CGAS) score is used to check the general 

#functioning level of children typically in the range of 1-100 

#CGAS score has max value 999 which might be an outlier.



##PCIAT-PCIAT_Total = Parent-Child Internet Addiction Test

#the score reflects addiction or PIU, which might be 0 if no problematic behavior is found

##SDS-SDS_Total_Raw: Self-Rating Depression , raw scores can range from: 20-80

#min for this is 38 in the table which means even lowest 

#scoring children may be experiencing mild depressive symptoms

##PreInt_EduHx-computerinternet_hoursday : avg number of hours per day spent using internet

In [9]:
####AG###



## Report of the extensive missing values ## 



# Fitness_Endurance-Max_Stage: 743 non-null (81% missing)

# Fitness_Endurance-Time_Mins: 740 non-null (81% missing)

# Fitness_Endurance-Time_Sec: 740 non-null (81% missing)

# Physical-Waist_Circumference: 898 non-null (77% missing)

# BIA-Activity_Level_num: 1991 non-null (50% missing)

# BIA-BMC: 1991 non-null (50% missing)

# BIA-BMI: 1991 non-null (50% missing)

# BIA-BMR: 1991 non-null (50% missing)

# BIA-DEE: 1991 non-null (50% missing)

# BIA-ECW: 1991 non-null (50% missing)

# PAQ_A-Season: 475 non-null (88% missing)

# PAQ_A-PAQ_A_Total: 475 non-null (88% missing)

# PAQ_C-Season: 1721 non-null (57% missing)

# PAQ_C-PAQ_C_Total: 1721 non-null (57% missing)

# PCIAT-Season: 2736 non-null (31% missing) with multiple related PCIAT columns also having missing data.

# SDS-Season: 2618 non-null (34% missing)

# SDS-SDS_Total_Raw: 2609 non-null (35% missing)

# SDS-SDS_Total_T: 2606 non-null (35% missing)



# CP: Check for missing/null values

train.isnull().sum()

id                                           0
Basic_Demos-Enroll_Season                    0
Basic_Demos-Age                              0
Basic_Demos-Sex                              0
CGAS-Season                               1405
CGAS-CGAS_Score                           1539
Physical-Season                            650
Physical-BMI                               938
Physical-Height                            933
Physical-Weight                            884
Physical-Waist_Circumference              3062
Physical-Diastolic_BP                     1006
Physical-HeartRate                         993
Physical-Systolic_BP                      1006
Fitness_Endurance-Season                  2652
Fitness_Endurance-Max_Stage               3217
Fitness_Endurance-Time_Mins               3220
Fitness_Endurance-Time_Sec                3220
FGC-Season                                 614
FGC-FGC_CU                                1638
FGC-FGC_CU_Zone                           1678
FGC-FGC_GSND 

In [10]:
# CP: Explore data

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 82 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      3960 non-null   object 
 1   Basic_Demos-Enroll_Season               3960 non-null   object 
 2   Basic_Demos-Age                         3960 non-null   int64  
 3   Basic_Demos-Sex                         3960 non-null   int64  
 4   CGAS-Season                             2555 non-null   object 
 5   CGAS-CGAS_Score                         2421 non-null   float64
 6   Physical-Season                         3310 non-null   object 
 7   Physical-BMI                            3022 non-null   float64
 8   Physical-Height                         3027 non-null   float64
 9   Physical-Weight                         3076 non-null   float64
 10  Physical-Waist_Circumference            898 non-null    floa

In [11]:
# CP: Explore data

train.shape

(3960, 82)

In [12]:
# CP: Check target values

train['sii'].value_counts()

sii
0.0    1594
1.0     730
2.0     378
3.0      34
Name: count, dtype: int64

In [13]:
# CP: Check missing target values

train['sii'].isnull().sum()

1224

In [14]:
# CP: Drop any rows where target value is missing

# since they cannot be used for training.

train.dropna(subset=['sii'], inplace=True)

In [15]:
# CP: Recheck missing target values

train['sii'].isnull().sum()

0

In [16]:
# CP: Check for duplicates

train.duplicated().sum()

0

In [29]:
train.shape

(2736, 82)

In [42]:
# 11/7/24 AG ###
#checking columns with more than 50% of the data missing
missing_percentage = train.isnull().mean() * 100
columns_with_missing_data = missing_percentage[missing_percentage > 50]
columns_with_missing_data


Physical-Waist_Circumference    82.346491
Fitness_Endurance-Season        53.947368
Fitness_Endurance-Max_Stage     73.282164
Fitness_Endurance-Time_Mins     73.391813
Fitness_Endurance-Time_Sec      73.391813
FGC-FGC_GSND                    68.128655
FGC-FGC_GSND_Zone               68.421053
FGC-FGC_GSD                     68.165205
FGC-FGC_GSD_Zone                68.421053
PAQ_A-Season                    86.732456
PAQ_A-PAQ_A_Total               86.732456
dtype: float64

In [58]:
#Checking the correlation of the numeric columns with sii 
numerical_columns = train.select_dtypes(include=['float64', 'int64']).columns

# No sii in  the numerical columns to avoid correlating it with itself
numerical_columns = numerical_columns[numerical_columns != 'sii']
correlation = train[numerical_columns].corrwith(train['sii'])

correlation


Basic_Demos-Age                           0.365990
Basic_Demos-Sex                          -0.100148
CGAS-CGAS_Score                          -0.085261
Physical-BMI                              0.221125
Physical-Height                           0.373530
Physical-Weight                           0.325938
Physical-Waist_Circumference              0.272885
Physical-Diastolic_BP                     0.050971
Physical-HeartRate                       -0.024572
Physical-Systolic_BP                      0.135336
Fitness_Endurance-Max_Stage              -0.070589
Fitness_Endurance-Time_Mins              -0.060628
Fitness_Endurance-Time_Sec                0.008701
FGC-FGC_CU                                0.247364
FGC-FGC_CU_Zone                          -0.023944
FGC-FGC_GSND                              0.149495
FGC-FGC_GSND_Zone                         0.001667
FGC-FGC_GSD                               0.163448
FGC-FGC_GSD_Zone                          0.023696
FGC-FGC_PU                     

In [59]:
train.isnull().sum()

id                                           0
Basic_Demos-Enroll_Season                    0
Basic_Demos-Age                              0
Basic_Demos-Sex                              0
CGAS-Season                                394
CGAS-CGAS_Score                            394
Physical-Season                            141
Physical-BMI                               209
Physical-Height                            206
Physical-Weight                            164
Physical-Waist_Circumference              2253
Physical-Diastolic_BP                      258
Physical-HeartRate                         250
Physical-Systolic_BP                       258
Fitness_Endurance-Season                  1476
Fitness_Endurance-Max_Stage               2005
Fitness_Endurance-Time_Mins               2008
Fitness_Endurance-Time_Sec                2008
FGC-Season                                  89
FGC-FGC_CU                                 817
FGC-FGC_CU_Zone                            852
FGC-FGC_GSND 

In [63]:
# Check the missing percentage for specific columns
columns_of_interest = ['PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'Physical-Waist_Circumference',
                       'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec']

# percentage of missing values in these columns
missing_percentage = train[columns_of_interest].isnull().mean() * 100
print("Missing percentage:")
print(missing_percentage)

numerical_columns = train[columns_of_interest].select_dtypes(include=['float64', 'int64']).columns

# Calculate correlation of the numerical columns with the target variable 'sii'
correlation_with_sii = train[numerical_columns].corrwith(train['sii'])

print("\nCorrelation with sii:")
print(correlation_with_sii)


Missing percentage:
PAQ_A-Season                    86.732456
PAQ_A-PAQ_A_Total               86.732456
Physical-Waist_Circumference    82.346491
Fitness_Endurance-Max_Stage     73.282164
Fitness_Endurance-Time_Mins     73.391813
Fitness_Endurance-Time_Sec      73.391813
dtype: float64

Correlation with sii:
PAQ_A-PAQ_A_Total              -0.050386
Physical-Waist_Circumference    0.272885
Fitness_Endurance-Max_Stage    -0.070589
Fitness_Endurance-Time_Mins    -0.060628
Fitness_Endurance-Time_Sec      0.008701
dtype: float64


In [64]:
#creating a backup
backup_train = train.copy()

In [76]:
#Dropping columns that have null values more than 50% and weak sii connection
#train.drop(columns=columns_of_interest, inplace=True)
train.isnull().sum()

id                                           0
Basic_Demos-Enroll_Season                    0
Basic_Demos-Age                              0
Basic_Demos-Sex                              0
CGAS-Season                                394
CGAS-CGAS_Score                            394
Physical-Season                            141
Physical-BMI                               209
Physical-Height                            206
Physical-Weight                            164
Physical-Diastolic_BP                      258
Physical-HeartRate                         250
Physical-Systolic_BP                       258
Fitness_Endurance-Season                  1476
FGC-Season                                  89
FGC-FGC_CU                                 817
FGC-FGC_CU_Zone                            852
FGC-FGC_GSND                              1864
FGC-FGC_GSND_Zone                         1872
FGC-FGC_GSD                               1865
FGC-FGC_GSD_Zone                          1872
FGC-FGC_PU   

In [88]:
train = train.fillna(0) # replacing null values with 0 for uniformity

In [93]:
train.isnull().sum()
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2736 entries, 0 to 3958
Data columns (total 76 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      2736 non-null   object 
 1   Basic_Demos-Enroll_Season               2736 non-null   object 
 2   Basic_Demos-Age                         2736 non-null   int64  
 3   Basic_Demos-Sex                         2736 non-null   int64  
 4   CGAS-Season                             2736 non-null   object 
 5   CGAS-CGAS_Score                         2736 non-null   float64
 6   Physical-Season                         2736 non-null   object 
 7   Physical-BMI                            2736 non-null   float64
 8   Physical-Height                         2736 non-null   float64
 9   Physical-Weight                         2736 non-null   float64
 10  Physical-Diastolic_BP                   2736 non-null   float64
 